Word2Vec and TextCNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
#from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
import numpy as np

# Sample data (replace with your dataset)
texts = ["This is a positive sentence.", "Negative sentiment here.", "Another positive example."]
labels = ["positive", "negative", "positive"]

# Tokenize and create Word2Vec embeddings
tokenized_texts = [text.lower().split() for text in texts]
word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=50, window=5, min_count=1, workers=4)
embedding_dim = word2vec_model.vector_size

# Convert texts to sequences of indices
X = torch.tensor([word2vec_model.wv[word] for sentence in tokenized_texts for word in sentence], dtype=torch.float32)
X = X.view(len(tokenized_texts), -1, embedding_dim)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
y = torch.tensor(y, dtype=torch.long)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the TextCNN model
class TextCNN(nn.Module): # Use PyTorch module to declare the class
    def __init__(self, embedding_dim, num_classes): # The features of the class's object - embedding dimension (how much can be understood) and the number of classes
        super(TextCNN, self).__init__()  # Get the inherited power from  PyTorch's TextCNN
        self.conv1 = nn.Conv1d(embedding_dim, 128, kernel_size=3) # Defining the convolution layer Understanding the embedding dim by focussing on the kernel size at once, using 128 strength
        self.fc1 = nn.Linear(128, num_classes) # making decision on the number of classes based on the 128 strength

    def forward(self, x): # Forwarding feeding training approch for class identification - input is the x
        x = self.conv1(x.permute(0, 2, 1)) # Con
        x = nn.functional.relu(x) # ReLu classifier algorithm
        x = nn.functional.max_pool1d(x, x.size(2)).squeeze(2) # Max pooling - feature selection for best prediction
        x = self.fc1(x) # Predict the class the input belong
        return x

# Instantiate the model, loss function, and optimizer
model = TextCNN(embedding_dim, num_classes=len(label_encoder.classes_))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = torch.sum(predicted == y_test).item() / len(y_test)
    print(f"Test Accuracy: {accuracy:.4f}")
